In [1]:
import numpy as np
import random
from collections import deque
import warnings
import numpy as np
import math

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from mdesc.models import (Eval, ClassifierEval, Sensitivity, ClassifierSensitivity)
from mdesc.data.datamanager import DataUtilities as DU

In [2]:
# initialize plotly notebook_mode
init_notebook_mode(connected=True)

In [3]:
import os
os.getcwd()

'/Users/jasonlewris/Desktop/GitProjects/model-describer'

In [14]:
# this is a test file
# sensitivity plot creation and testing

modelObjc = RandomForestRegressor()

# wine quality dataset example
wine = pd.read_csv('/Users/jasonlewris/Desktop/GitProjects/model-describer/debug/data/wine.csv')

groupby_df = wine[['alcohol', 'Type']]
groupby_df['all'] = 1

ydepend = 'quality'

X = wine.loc[:, wine.columns != ydepend]
y = wine.loc[:, ydepend]

#y = np.array(['bad' if val < 5 else 'good' for val in y])

X = pd.get_dummies(X)

modelObjc.fit(X, y)

RE = Sensitivity(prediction_fn=modelObjc.predict)
res = RE.fit_transform(X=X, y=y, groupby_df=groupby_df)


/Users/jasonlewris/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



0 alcohol
high


/Users/jasonlewris/Desktop/GitProjects/model-describer/mdesc/models.py:123: RuntimeWarning:

Mean of empty slice

/Users/jasonlewris/Desktop/GitProjects/model-describer/mdesc/models.py:124: RuntimeWarning:

Mean of empty slice



low
medium
1 Type
Red
White
2 all
1


In [15]:
RE.data_set.viz_now(groupby_name='alcohol')